In [1]:
%matplotlib inline
import nengo
import numpy as np
import scipy.special

When working with learning and neural networks, it is useful to be able to control the sparsity of the network.  In this case, I'll define sparsity to mean "the proportion of the input space that causes a particular neuron to fire".  So if a neuron has sparsity = 0.1, then it will fire for 10% of its inputs.

In the NEF, the only parameter we have to control sparsity is the $x_{intercept}$.  This is the value where if $x \cdot e > x_{intercept}$, then the neuron will fire.

So, we need to compute the $x_{intercept}$ that gives a certain level of sparsity.

To do this, let's consider a hyperspherical cap: https://en.wikipedia.org/wiki/Spherical_cap#Hyperspherical_cap

<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Spherical_cap_diagram.tiff/lossless-page1-220px-Spherical_cap_diagram.tiff.png>

The volume is $V = {1 \over 2} C_d r^d I_{2rh-h^2 \over r^2}({d+1 \over 2}, {1 \over 2})$ where $C_d$ is the volume of a unit hyperball of dimension $d$ and $I_x(a,b)$ is the regularized incomplete beta function.

The surface area is $A = {1 \over 2} A_d r^d I_{2rh-h^2 \over r^2}({d-1 \over 2}, {1 \over 2})$ where $A_d$ is the surface area of a unit hypersphere of dimension $d$ and $I_x(a,b)$ is the regularized incomplete beta function.

When we're working with SSPs, we are dealing with points on the surface of a hypersphere of radius 1, and $h=1-x_{intercept}$.  So, the proportion of points inside the hyperspeherical cap is $A/A_d$, which is

$p = {1 \over 2} I_{1-{x_{intercept}^2}}({{d-1} \over 2}, {1 \over 2})$

If we have this proportion, but we want to compute the $x_{intercept}$, then we need to invert this function:

$2p = I_{1-{x_{intercept}^2}}({{d-1} \over 2}, {1 \over 2})$

$1-{x_{intercept}^2} = I^{-1}_{2p}({{d-1} \over 2}, {1 \over 2})$

${x_{intercept}^2} = 1-I^{-1}_{2p}({{d-1} \over 2}, {1 \over 2})$

$x_{intercept} = \sqrt{1-I^{-1}_{2p}({{d-1} \over 2}, {1 \over 2})}$



Of course, this formula only works for $p<0.5$.  For $p>0.5$ we can use 1-x_{intercept} and flip the sign.

In [2]:
def sparsity_to_x_intercept(d, p):
    sign = 1
    if p > 0.5:
        p = 1.0 - p
        sign = -1
    return sign * np.sqrt(1-scipy.special.betaincinv((d-1)/2.0, 0.5, 2*p))


One thing to note is that if we want the same thing but for *volume* (i.e. for representing points that are inside the hypersphere), then we can do the same derivation but using the volume formula.  The only difference is that instead of d-1, you get d+1.  The d+1 version of this formula is what I used for the original derivation of intercepts that lead to the CosineSimilarity(D-2) suggestion for initializing intercepts (if you want a uniform distribution of sparsity).  For that derivation, see https://github.com/tcstewar/testing_notebooks/blob/master/Intercept%20Distribution%20.ipynb

Let's test this formula.  We'll do it by generating a neuron with the intercept computed with this function, and measuring its sparsity by randomly sampling points on the surface of the hypersphere.

In [3]:
D = 32
N = 1000000
sparsity = 0.1
intercept = sparsity_to_x_intercept(D, sparsity)

model = nengo.Network()
with model:
    ens = nengo.Ensemble(n_neurons=1, dimensions=D,
                         intercepts=[intercept])
sim = nengo.Simulator(model)

# generate samples just on the surface of the sphere
pts = nengo.dists.UniformHypersphere(surface=True).sample(N, D)

_, A = nengo.utils.ensemble.tuning_curves(ens, sim, inputs=pts)
    
print('Computed sparsity:', np.mean(A>0))

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

Computed sparsity: 0.100237
